# 第5章 决策树

参考：https://github.com/fengdu78/lihang-code

1．分类决策树模型是表示基于特征对实例进行分类的树形结构。决策树可以转换成一个**if-then**规则的集合，也可以看作是定义在特征空间划分上的类的条件概率分布。

2．决策树学习旨在构建一个与训练数据拟合很好，并且复杂度小的决策树。因为从可能的决策树中直接选取最优决策树是NP完全问题。现实中采用启发式方法学习次优的决策树。

决策树学习算法包括3部分：特征选择、树的生成和树的剪枝。常用的算法有ID3、
C4.5和CART。

3．特征选择的目的在于选取对训练数据能够分类的特征。特征选择的关键是其准则。常用的准则如下：

（1）样本集合$D$对特征$A$的信息增益（ID3）


$$g(D, A)=H(D)-H(D|A)$$

$$H(D)=-\sum_{k=1}^{K} \frac{\left|C_{k}\right|}{|D|} \log _{2} \frac{\left|C_{k}\right|}{|D|}$$

$$H(D | A)=\sum_{i=1}^{n} \frac{\left|D_{i}\right|}{|D|} H\left(D_{i}\right)$$

其中，$H(D)$是数据集$D$的熵，$H(D_i)$是数据集$D_i$的熵，$H(D|A)$是数据集$D$对特征$A$的条件熵。	$D_i$是$D$中特征$A$取第$i$个值的样本子集，$C_k$是$D$中属于第$k$类的样本子集。$n$是特征$A$取 值的个数，$K$是类的个数。

（2）样本集合$D$对特征$A$的信息增益比（C4.5）


$$g_{R}(D, A)=\frac{g(D, A)}{H(D)}$$


其中，$g(D,A)$是信息增益，$H(D)$是数据集$D$的熵。

（3）样本集合$D$的基尼指数（CART）

$$\operatorname{Gini}(D)=1-\sum_{k=1}^{K}\left(\frac{\left|C_{k}\right|}{|D|}\right)^{2}$$

特征$A$条件下集合$D$的基尼指数：

 $$\operatorname{Gini}(D, A)=\frac{\left|D_{1}\right|}{|D|} \operatorname{Gini}\left(D_{1}\right)+\frac{\left|D_{2}\right|}{|D|} \operatorname{Gini}\left(D_{2}\right)$$
 
4．决策树的生成。通常使用信息增益最大、信息增益比最大或基尼指数最小作为特征选择的准则。决策树的生成往往通过计算信息增益或其他指标，从根结点开始，递归地产生决策树。这相当于用信息增益或其他准则不断地选取局部最优的特征，或将训练集分割为能够基本正确分类的子集。

5．决策树的剪枝。由于生成的决策树存在过拟合问题，需要对它进行剪枝，以简化学到的决策树。决策树的剪枝，往往从已生成的树上剪掉一些叶结点或叶结点以上的子树，并将其父结点或根结点作为新的叶结点，从而简化生成的决策树。


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from collections import Counter
import math
from math import log
import pprint

## 书上题目5.1

In [23]:
def create_data():
    datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    feature_names = ['年龄', '有工作', '有自己的房子', '信贷情况', '类别']
    # 返回数据集和每个维度的名称
    return datasets, feature_names

In [24]:
datasets, feature_names = create_data()

In [25]:
train_data = pd.DataFrame(datasets, columns=feature_names)

In [26]:
train_data

,年龄,有工作,有自己的房子,信贷情况,类别
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


In [15]:

# 定义树节点类, 分类树
class Node:
    def __init__(self, root=True, label=None, feature_name=None, feature=None):
        # root 标记是否为单结点树（即叶子结点）
        self.root = root
        # label 为当前节点的类别标记
        self.label = label
        # feature_name 为当前节点的特征名
        self.feature_name = feature_name
        # 子树
        self.tree = {}
        self.result = {
            'lable': self.label,
            'feature': self.feature_name,
            'tree': self.tree
        }

    def __repr__(self):
        """
        重写 __repr__ 用于显示对象，也可以 使用 __str__
        dt = Node()
        print(dt) 就可以显示自己定义的信息了
        这里  self.result 中包含 self.tree，所以会递归显示所有子树信息
        :return:
        """
        return '{}'.format(self.result)

    def add_node(self, val, node):
        """
        增加树节点
        """
        self.tree[val] = node

    def predict(self, features):
        if self.root is True:
            return self.label
        return self.tree[features[self.feature]].predict(features)


class DTree:
    def __init__(self, epsilon=0.1):
        self.epsilon = epsilon
        self._tree = {}

    # 熵
    @staticmethod
    def calc_ent(datasets):
        data_length = len(datasets)
        label_count = {}
        for i in range(data_length):
            label = datasets[i][-1]
            if label not in label_count:
                label_count[label] = 0
            label_count[label] += 1
        ent = -sum([(p / data_length) * log(p / data_length, 2)
                    for p in label_count.values()])
        return ent

    # 经验条件熵 H(D|A)
    def cond_ent(self, datasets, index=0):
        """
        :param index: 当前特征列 A 在 datasets 中的索引
        """
        data_length = len(datasets)
        # 其中每个元素存储当前特征值时的所有样本
        feature_sets = {}
        for i in range(data_length):
            feature = datasets[i][index]
            if feature not in feature_sets:
                feature_sets[feature] = []
            # 当前样本 datasets[i] 加入列表中
            feature_sets[feature].append(datasets[i])
        # H(D|A)
        cond_ent = sum([(len(p) / data_length) * self.calc_ent(p) for p in feature_sets.values()])
        return cond_ent

    # 信息增益
    @staticmethod
    def info_gain(ent, cond_ent):
        return ent - cond_ent

    def info_gain_train(self, datasets):
        count = len(datasets[0]) - 1
        ent = self.calc_ent(datasets)
        best_feature = []
        # 对所有特征计算信息增益
        for c in range(count):
            c_info_gain = self.info_gain(ent, self.cond_ent(datasets, index=c))
            best_feature.append((c, c_info_gain))

        # 选择信息增益最大的特征作为根节点
        best_ = max(best_feature, key=lambda x: x[-1])
        return best_

    def train(self, train_data):
        """
        :param train_data: 训练数据， pandas 格式，最后一列为 label
        :return: 决策树
        """
        # y_train: 最后一列 label ， features : 特征名
        _, y_train, features = train_data.iloc[:, :-1], train_data.iloc[:, -1], train_data.columns[: -1]

        # 1. 若 D 中实例属于同一类 Ck， 则 T 为单节点树，并将类 Ck 作为结点的类标记，返回T
        if len(y_train.value_counts()) == 1:
            return Node(root=True, label=y_train.iloc[0])
        # 2. 若特征集 A 为空，则T为单节点树，将 D 中实例数最大的类 Ck 作为该结点的类标记，返回 T
        if len(features) == 0:
            # 对 Series 排序，去索引第一的作为类标
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])
        # 3. 计算 A 中各特征对 D 的最大信息增益，选择 Ag
        max_feature, max_info_gain = self.info_gain_train(np.array(train_data))
        max_feature_name = features[max_feature]

        # 4. Ag 的信息增益小于阈值 eta，则置T为单节点树，并将D中是实例数最大的类Ck作为该节点的类标记，返回T
        if max_info_gain < self.epsilon:
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])

        # 5. 构建 Ag 子集，对每一个 Ag = ai 将D分割为若干个子集（相当于多个子树），子集中实例数最大的类别作为当前节点
        node_tree = Node(root=False, feature_name=max_feature_name)
        # 获取特征 Ag 的所有取值
        feature_list = train_data[max_feature_name].unique()
        for f in feature_list:
            # 获取子集，删掉当前特征
            sub_train_df = train_data.loc[train_data[max_feature_name] == f].drop([max_feature_name], axis=1)

            # 6. 递归调用，生成树
            sub_tree = self.train(sub_train_df)
            node_tree.add_node(f, sub_tree)

        return node_tree

    def fit(self, train_data, label):
        train_data['label'] = label
        self._tree = self.train(train_data)
        return self._tree

    def predict(self, X_test):
        return self._tree.predict(X_test)

    def score(self, test_data, test_label):
        """
        :param test_data: 测试数据
        :param test_label: 测试数据标签
        :return:
        """
        predict_label = self.predict(test_data)

        error_cnt = 0
        for k in range(len(predict_label)):
            # 记录误分类数
            if predict_label[k] != test_label[k]:
                error_cnt += 1
        # 正确率 = 1 - 错误分类样本数 / 总样本数
        print('accuracy: ', 1 - error_cnt / len(predict_label))

In [16]:
info_gain_train(np.array(datasets), labels)

特征(年龄) - info_gain - 0.083
特征(有工作) - info_gain - 0.324
特征(有自己的房子) - info_gain - 0.420
特征(信贷情况) - info_gain - 0.363
(2, 0.4199730940219749)


'特征(有自己的房子)的信息增益最大，选择为根节点特征'

## 利用ID3算法生成决策树，例5.3

In [56]:

# 定义树节点类, 分类树
class Node:
    def __init__(self, root=True, label=None, feature_name=None):
        # root 标记是否为单结点树（即叶子结点）
        self.root = root
        # label 为当前节点的类别标记
        self.label = label
        # feature_name 为当前节点的特征名
        self.feature_name = feature_name

        # 子树，字典形式，（特征取值：子树）
        self.tree = {}
        self.result = {
            'lable': self.label,
            'feature': self.feature_name,
            'tree': self.tree
        }

    def __repr__(self):
        return '{}'.format(self.result)

    def add_node(self, val, node):
        """
        :param val: 特征的取值
        :param node:
        :return:
        """
        """
        增加树节点
        """
        self.tree[val] = node

    def predict(self, x_sample):
        """
        :param x_sample: 一行数据 Series 格式，能通过特征名索引特征值
        :return: 当前样本的预测值
        """
        if self.root is True:
            return self.label
        # x_sample[self.feature_name] 当前特征下样本实例的取值
        return self.tree[x_sample[self.feature_name]].predict(x_sample)


class DTree:
    def __init__(self, epsilon=0.1):
        self.epsilon = epsilon
        self._tree = {}

    # 熵
    @staticmethod
    def calc_ent(datasets):
        data_length = len(datasets)
        label_count = {}
        for i in range(data_length):
            label = datasets[i][-1]
            if label not in label_count:
                label_count[label] = 0
            label_count[label] += 1
        ent = -sum([(p / data_length) * log(p / data_length, 2)
                    for p in label_count.values()])
        return ent

    # 经验条件熵 H(D|A)
    def cond_ent(self, datasets, index=0):
        """
        :param index: 当前特征列 A 在 datasets 中的索引
        """
        data_length = len(datasets)
        # 其中每个元素存储当前特征值时的所有样本
        feature_sets = {}
        for i in range(data_length):
            feature = datasets[i][index]
            if feature not in feature_sets:
                feature_sets[feature] = []
            # 当前样本 datasets[i] 加入列表中
            feature_sets[feature].append(datasets[i])
        # H(D|A)
        cond_ent = sum([(len(p) / data_length) * self.calc_ent(p) for p in feature_sets.values()])
        return cond_ent

    # 信息增益
    @staticmethod
    def info_gain(ent, cond_ent):
        return ent - cond_ent

    def info_gain_train(self, datasets):
        count = len(datasets[0]) - 1
        ent = self.calc_ent(datasets)
        best_feature = []
        # 对所有特征计算信息增益
        for c in range(count):
            c_info_gain = self.info_gain(ent, self.cond_ent(datasets, index=c))
            best_feature.append((c, c_info_gain))

        # 选择信息增益最大的特征作为根节点
        best_ = max(best_feature, key=lambda x: x[-1])
        return best_

    def train(self, train_data):
        """
        :param train_data: 训练数据， pandas 格式，最后一列为 label
        :return: 决策树
        """
        # y_train: 最后一列 label ， features : 特征名
        _, y_train, features = train_data.iloc[:, :-1], train_data.iloc[:, -1], train_data.columns[: -1]

        # 1. 若 D 中实例属于同一类 Ck， 则 T 为单节点树，并将类 Ck 作为结点的类标记，返回T
        if len(y_train.value_counts()) == 1:
            return Node(root=True, label=y_train.iloc[0])
        # 2. 若特征集 A 为空，则T为单节点树，将 D 中实例数最大的类 Ck 作为该结点的类标记，返回 T
        if len(features) == 0:
            # 对 Series 排序，去索引第一的作为类标
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])
        # 3. 计算 A 中各特征对 D 的最大信息增益，选择 Ag
        max_feature, max_info_gain = self.info_gain_train(np.array(train_data))
        max_feature_name = features[max_feature]

        # 4. Ag 的信息增益小于阈值 eta，则置T为单节点树，并将D中是实例数最大的类Ck作为该节点的类标记，返回T
        if max_info_gain < self.epsilon:
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])

        # 5. 构建 Ag 子集，对每一个 Ag = ai 将D分割为若干个子集（相当于多个子树），子集中实例数最大的类别作为当前节点
        node_tree = Node(root=False, feature_name=max_feature_name)
        # 获取特征 Ag 的所有取值
        feature_list = train_data[max_feature_name].unique()
        for f in feature_list:
            # 获取子集，删掉当前特征
            sub_train_df = train_data.loc[train_data[max_feature_name] == f].drop([max_feature_name], axis=1)

            # 6. 递归调用，生成树
            sub_tree = self.train(sub_train_df)
            node_tree.add_node(f, sub_tree)

        return node_tree

    def fit(self, train_data):
        self._tree = self.train(train_data)
        return self._tree

    def predict_one_sample(self, x_vec):
        """
        预测一条样本
        :param x_vec: 样本向量
        :return:
        """
        return self._tree.predict(x_vec)

    def predict(self, X_test):
        """
        预测结果
        :param X_test: DataFrame 格式
        :return:
        """
        result = []
        for i in range(X_test.shape[0]):
            result.append(self.predict_one_sample(X_test.iloc[i]))
        return result

    def score(self, test_data, test_label):
        """
        :param test_data: 测试数据
        :param test_label: 测试数据标签
        :return:
        """
        predict_label = self.predict(test_data)

        error_cnt = 0
        for k in range(len(predict_label)):
            # 记录误分类数
            if predict_label[k] != test_label[k]:
                error_cnt += 1
        # 正确率 = 1 - 错误分类样本数 / 总样本数
        print('accuracy: ', 1 - error_cnt / len(predict_label))

def create_data():
    datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    feature_names = ['年龄', '有工作', '有自己的房子', '信贷情况', '类别']
    # 返回数据集和每个维度的名称
    return datasets, feature_names

In [59]:
datasets, labels = create_data()
data_df = pd.DataFrame(datasets, columns=labels)
dt = DTree()
tree = dt.fit(data_df)
print(tree)
X_train = data_df.iloc[:, :-1]
y_train = data_df.iloc[:, -1]
dt.score(X_train, y_train)

{'lable': None, 'feature': '有自己的房子', 'tree': {'否': {'lable': None, 'feature': '有工作', 'tree': {'否': {'lable': '否', 'feature': None, 'tree': {}}, '是': {'lable': '是', 'feature': None, 'tree': {}}}}, '是': {'lable': '是', 'feature': None, 'tree': {}}}}
accuracy:  1.0


In [63]:
print(X_train.iloc[5])
print("label:" + y_train[5])
print("predict:" + dt.predict_one_sample(X_train.iloc[5]))

年龄        中年
有工作        否
有自己的房子     否
信贷情况      一般
Name: 5, dtype: object
label:否
predict:否


In [65]:
a = np.array([[1,2],[3,4]])

In [66]:
print(pd.DataFrame(a))

   0  1
0  1  2
1  3  4
